In [1]:
import json
import os
import random
import numpy as np

random.seed(109)
np.random.seed(109)

In [2]:
root = '/Users/tokkuman/git-tokkuman/embryo_classification/datasets'
with open(os.path.join(root, 'labels', 'born.txt'), 'r') as f:
    born_list = [line.rstrip() for line in f]
with open(os.path.join(root, 'labels', 'abort.txt'), 'r') as f:
    abort_list = [line.rstrip() for line in f]
    
print(born_list)
print(abort_list)
print(len(born_list), len(abort_list))

['E03F02Embryo03', 'E03F04Embryo04', 'E03F06Embryo02', 'E03F09Embryo01', 'E03F10Embryo04', 'E03F11Embryo01', 'E03F13Embryo03', 'E04F02Embryo01', 'E04F03Embryo01', 'E04F07Embryo01', 'E04F07Embryo03', 'E04F08Embryo03', 'E04F12Embryo04', 'E04F13Embryo04', 'E05F01Embryo04', 'E05F05Embryo01', 'E05F06Embryo03', 'E05F06Embryo04', 'E05F08Embryo01', 'E05F08Embryo03', 'E05F09Embryo03', 'E05F12Embryo01', 'E05F12Embryo02', 'E05F12Embryo04', 'E05F15Embryo02', 'E06F02Embryo01', 'E06F03Embryo01', 'E06F03Embryo03', 'E06F04Embryo02', 'E06F06Embryo01', 'E06F07Embryo03', 'E06F09Embryo04', 'E06F10Embryo01', 'E06F10Embryo02', 'E06F12Embryo01', 'E06F13Embryo02', 'E06F13Embryo04', 'E06F14Embryo04', 'E11F01Embryo03', 'E11F04Embryo04', 'E11F06Embryo02', 'E11F09Embryo01', 'E11F14Embryo06', 'E11F15Embryo02', 'E11F19Embryo02', 'E11F19Embryo03', 'E11F20Embryo05', 'E12F03Embryo01', 'E12F06Embryo06', 'E12F07Embryo05', 'E12F10Embryo02', 'E13F01Embryo02', 'E13F01Embryo03', 'E13F01Embryo04', 'E13F01Embryo06', 'E13F07Em

# make split list for k-fold cross-validation

In [3]:
random.shuffle(born_list)
random.shuffle(abort_list)
train = 0.6
val = 0.2
test = 0.2

os.makedirs(os.path.join(root, 'split_list'), exist_ok=True)
os.makedirs(os.path.join(root, 'split_list', 'kfcv'), exist_ok=True)
dist = os.path.join(root, 'split_list', 'kfcv', 'test.txt')
with open(dist, 'w') as f:
    for i in range(20, 29):
        f.write('{}\n'.format(abort_list[i]))
    for i in range(40, 62):
        f.write('{}\n'.format(born_list[i]))

for v in range(4):
    dist = os.path.join(root, 'split_list', 'kfcv', 'set{0:02d}'.format(v+1))
    os.makedirs(dist, exist_ok=True)
    with open(os.path.join(dist, 'validation.txt'), 'w') as f:
        for i in range(v*10, (v+1)*10):
            f.write('{}\n'.format(born_list[i]))
        for i in range(v*5, (v+1)*5):
            f.write('{}\n'.format(abort_list[i]))

    with open(os.path.join(dist, 'train.txt'), 'w') as f:
        for i in range(0, v*10):
            f.write('{}\n'.format(born_list[i]))
        for i in range(0, v*5):
            f.write('{}\n'.format(abort_list[i]))
        for i in range((v+1)*10, 40):
            f.write('{}\n'.format(born_list[i]))
        for i in range((v+1)*5, 20):
            f.write('{}\n'.format(abort_list[i]))


# make split list for monte-carlo cross-validation

In [4]:
fold = 15

### [ratio]  train : val : test = 6 : 2 : 2
### [number] train : val : test = 39 : 12 : 14
num_born_train = 36
num_born_val = 13
num_born_test = 13
num_abort_train = 17
num_abort_val = 6
num_abort_test = 6

os.makedirs(os.path.join(root, 'split_list'), exist_ok=True)
os.makedirs(os.path.join(root, 'split_list', 'mccv'), exist_ok=True)
for v in range(1, fold+1):
    random.shuffle(born_list)
    random.shuffle(abort_list)
    dist = os.path.join(root, 'split_list', 'mccv', 'set{0:02d}'.format(v))
    os.makedirs(dist, exist_ok=True)

    with open(os.path.join(dist, 'test.txt'), 'w') as f:
        for i in range(num_born_test):
            f.write('{}\n'.format(born_list[i]))
        for i in range(num_abort_test):
            f.write('{}\n'.format(abort_list[i]))

    with open(os.path.join(dist, 'train.txt'), 'w') as f:
        for i in range(num_born_test, num_born_test + num_born_train):
            f.write('{}\n'.format(born_list[i]))
        for i in range(num_abort_test, num_abort_test + num_abort_train):
            f.write('{}\n'.format(abort_list[i]))
            
    with open(os.path.join(dist, 'validation.txt'), 'w') as f:
        for i in range(num_born_test + num_born_train, num_born_test + num_born_train + num_born_val):
            f.write('{}\n'.format(born_list[i]))
        for i in range(num_abort_test + num_abort_train, num_abort_test + num_abort_train + num_abort_val):
            f.write('{}\n'.format(abort_list[i]))